

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_GM_DE.ipynb)




# **ICD10 coding for German**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## 1. Colab Setup

Import license keys

In [1]:
import os
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

Saving v3_spark_nlp_for_healthcare.json to v3_spark_nlp_for_healthcare.json
SparkNLP Version: 3.0.1
SparkNLP-JSL Version: 3.0.0


Install dependencies

In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

# Install Spark NLP Display for visualization
!pip install --ignore-installed spark-nlp-display

Import dependencies into Python

In [3]:
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

Start the Spark session

In [4]:
spark = sparknlp_jsl.start(license_keys['SECRET'])

# manually start session
# params = {"spark.driver.memory" : "16G",
#           "spark.kryoserializer.buffer.max" : "2000M",
#           "spark.driver.maxResultSize" : "2000M"}

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

## 2. Construct the pipeline

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer()\
    .setInputCols(['sentences']) \
    .setOutputCol('tokens')

embeddings = WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained('ner_healthcare_slim', "de", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"])\
    .setOutputCol("ner_tags")

#Filtering NER chunks as ICD codes only relate to specific entities.
ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunk").setWhiteList(['MEDICAL_CONDITION'])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

entity_resolver = ChunkEntityResolverModel\
    .pretrained("chunkresolve_ICD10GM_2021","de","clinical/models")\
    .setInputCols("tokens","chunk_embeddings").setOutputCol("resolution")
 
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_chunker,
    chunk_embeddings,
    entity_resolver])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = pipeline.fit(empty_df)

light_pipeline = sparknlp.base.LightPipeline(pipeline_model)

w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare_slim download started this may take some time.
Approximate size to download 14.2 MB
[OK!]
chunkresolve_ICD10GM_2021 download started this may take some time.
Approximate size to download 49.4 MB
[OK!]


## 3. Create example inputs

In [7]:
# Enter examples as strings in this array
input_list = [
     """Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ich seit dem nicht mehr losgeworden bin. Vom Symbioflor2 bekam ich dann auch immer stärkere körperliche Beschwerden, so daß ich es mit der Angst zu tun bekam und meinen Arzt bat mir Cotrim zuverschreiben (das zweitemal nach 1983). Die Folge war leider, daß ich starken Fußschweiß bekam mit entsprechendem Geruch. Auch meine Hände fingen stark an zu riechen.
Ich war im Schlaflabor. Nur eine Auffälligkeit. In der ersten Nacht dort trat ich verfrüht in den Traumschlaf ein, was ein Kriterium für Narkolepsie ist. Bluttest und Gentest aber waren negativ. Meine Lungen sind super, mein MRT unauffällig. Jetzt bin ich in einer neurologischen Spezialklinik. Der Arzt meinte, dass eine sehr geringe Wahrscheinlichkeit besteht, dass ich doch an Narkolepsie leide. Ich soll in einem halben Jahr nochmal dort ins Schlaflabor. Medikamente gegen die Müdigkeit will er mir nicht verschreiben. Antidepressiva wären es, aber die müsste ich mir abgewöhnen, bevor ich in das Schlaflabor gehe. Also lieber "clean" bleiben.
Ich glaube mittlerweile, dass das Problem ganz woanders liegt... obwohl ich als Kind laut Aussagen meiner Eltern immer recht schnell dabei war, was laufen, Fahrradfahren etc. angeht, habe ich jetzt gerade als Erwachsener gemerkt, dass ich sehr langsam dabei bin, komplexe Bewegungen einzulernen und richtig auszuführen, sei es z.B. Tanzen oder Kampfsport. Wobei das sicherlich was ist, was mit der Zeit und Training schon deutlich besser geworden ist.""",
]

# 4. Run the pipeline

In [8]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)
light_result = light_pipeline.fullAnnotate(input_list[0])

# 5. Visualize

Full Pipeline

In [9]:
result.select(
    F.explode(
        F.arrays_zip('ner_chunk.result', 
                     'ner_chunk.begin',
                     'ner_chunk.end',
                     'ner_chunk.metadata',
                     'resolution.metadata', 'resolution.result')
    ).alias('cols')
).select(
    F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']").alias('begin'),
    F.expr("cols['2']").alias('end'),
    F.expr("cols['3']['entity']").alias('entity'),
    F.expr("cols['4']['resolved_text']").alias('icd10_description'),
    F.expr("cols['5']").alias('icd10_code'),
).show(truncate=False)

+-------------+-----+---+-----------------+------------------------+----------+
|chunk        |begin|end|entity           |icd10_description       |icd10_code|
+-------------+-----+---+-----------------+------------------------+----------+
|Beschwerden  |199  |209|MEDICAL_CONDITION|Dyspeptische Beschwerden|R10.1     |
|Angst        |234  |238|MEDICAL_CONDITION|Angst                   |F41.9     |
|Fußschweiß   |364  |373|MEDICAL_CONDITION|Fußschweiß              |R61.0     |
|Auffälligkeit|486  |498|MEDICAL_CONDITION|Augenlidsyringom        |D23.1     |
|Narkolepsie  |590  |600|MEDICAL_CONDITION|Narkolepsie             |G47.4     |
|Narkolepsie  |833  |843|MEDICAL_CONDITION|Narkolepsie             |G47.4     |
|Müdigkeit    |934  |942|MEDICAL_CONDITION|Müdigkeit               |R53       |
+-------------+-----+---+-----------------+------------------------+----------+



Light Pipeline

In [11]:
from sparknlp_display import EntityResolverVisualizer

vis = EntityResolverVisualizer()

## To set custom label colors:
vis.set_label_colors({'TREATMENT':'#800080', 'PROBLEM':'#77b5fe'})

vis.display(light_result[0], 'ner_chunk', 'resolution', 'document')